In [1]:
!pip install facebook-sdk



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import facebook
import json
from datetime import datetime
import time
from collections import defaultdict
import pandas as pd

In [3]:
import facebook  # Thư viện để tương tác với Facebook Graph API
import pandas as pd  # Thư viện để làm việc với dữ liệu dạng bảng (DataFrame)
import json  # Thư viện để xử lý dữ liệu JSON
from datetime import datetime  # Thư viện để làm việc với ngày giờ
import os  # Thư viện để làm việc với hệ thống tệp và thư mục


class FacebookCollector:
    def __init__(self, access_token):
        """
        Khởi tạo đối tượng FacebookCollector với mã thông báo truy cập (access token) đã cung cấp.
        """
        try:
            self.access_token = access_token  # Lưu mã thông báo truy cập
            self.graph = facebook.GraphAPI(access_token)  # Khởi tạo đối tượng GraphAPI để tương tác với Facebook
            print("FacebookCollector initialized successfully.")
        except Exception as e:
            print(f"Initialization error: {e}")  # In ra lỗi nếu không thể khởi tạo đối tượng

    def check_token_validity(self):
        """
        Kiểm tra xem mã thông báo truy cập có hợp lệ hay không.
        """
        try:
            # Lấy thông tin người dùng hiện tại để kiểm tra tính hợp lệ của mã thông báo
            me = self.graph.get_object('me', fields='id,name')
            print(f"Token is valid. User: {me.get('name')}, ID: {me.get('id')}")
            return True
        except Exception as e:
            print(f"Invalid token: {e}")  # In lỗi nếu mã thông báo không hợp lệ
            return False

    def collect_data(self, limit=10):
        """
        Thu thập bài viết từ dòng thời gian của người dùng Facebook (feed) với giới hạn bài viết (limit).
        """
        try:
            # Lấy các bài viết từ dòng thời gian của người dùng
            posts = self.graph.get_object(
                "me/feed",  # Lấy dữ liệu từ dòng thời gian của người dùng hiện tại
                fields="id,message,created_time,comments.limit(100),reactions.limit(100),shares,type",  # Chọn các trường cần thu thập
                limit=limit  # Giới hạn số lượng bài viết lấy về
            )
            print("Posts collected successfully:")
            # Lặp qua từng bài viết và in ra thông tin chi tiết
            for post in posts.get('data', []):
                print(f"Id: {post.get('id')}")
                print(f"Message: {post.get('message')}")
                print(f"Created Time: {post.get('created_time')}")
                print(f"Comments: {post.get('comments')}")
                print(f"Reactions: {post.get('reactions')}")
                print(f"Shares: {post.get('shares')}")
                print(f"Type: {post.get('type')}")
                print('______________________')
            return posts  # Trả về dữ liệu bài viết thu thập được
        except Exception as e:
            print(f"Error in collect_data: {e}")  # In lỗi nếu có vấn đề trong quá trình thu thập dữ liệu
            return None  # Trả về None nếu thu thập dữ liệu không thành công

    def json_to_excel(self, my_json, excel_file=None, output_dir=None):
        """
        Chuyển dữ liệu JSON sang file Excel và lưu vào thư mục đầu ra được chỉ định.
        """
        try:
            # Chuẩn bị dữ liệu cho DataFrame
            posts = []
            for post in my_json.get('data', []):
                post_data = {
                    'id': post.get('id', ''),  # Lấy ID bài viết
                    'message': post.get('message', ''),  # Lấy nội dung bài viết
                    'created_time': post.get('created_time', '')  # Lấy thời gian tạo bài viết
                }
                posts.append(post_data)  # Thêm thông tin bài viết vào danh sách

            # Tạo DataFrame từ danh sách bài viết
            df = pd.DataFrame(posts)
            if 'created_time' in df.columns:
                # Chuyển đổi cột 'created_time' thành định dạng ngày giờ (YYYY-MM-DD HH:MM:SS)
                df['created_time'] = pd.to_datetime(df['created_time']).dt.strftime('%Y-%m-%d %H:%M:%S')

            # Xác định tên file Excel và thư mục lưu
            if not excel_file:
                timestamp = datetime.now().strftime('%Y%m%d%H%M%S')  # Tạo tên file với dấu thời gian
                excel_file = f'facebook_posts_{timestamp}.xlsx'
            
            # Đảm bảo thư mục đầu ra tồn tại
            if output_dir:
                os.makedirs(output_dir, exist_ok=True)  # Tạo thư mục nếu chưa tồn tại
                excel_file = os.path.join(output_dir, excel_file)  # Lưu đường dẫn đầy đủ

            # Xuất dữ liệu DataFrame vào file Excel
            df.to_excel(excel_file, index=False)
            print(f"Data saved to {excel_file}")  # In thông báo nếu lưu thành công
            return excel_file  # Trả về đường dẫn file Excel
        except Exception as e:
            print(f"Error exporting to Excel: {e}")  # In lỗi nếu có vấn đề trong quá trình lưu dữ liệu
            return None  # Trả về None nếu xuất dữ liệu không thành công

def main():
    ACCESS_TOKEN = 'EAASkTBte5fYBO3KXGrlEjvU8DV7ttxDKnKK5ZApsW5YUBD8JpUXZCMhl8CZCPUOJMhPGn1GFLSov74f3b1Q8GjoINNhc0iC38newyP91Gom1kAJ0OmHOVND1mRnLVOL5VkY5aYw4VZA4ggyZCnI1aE9L2AEfLX6rnbIR19c73SFnZCWqXH9QOoXDaJONeJHQ8lB2m0PZCRcwesIoeDWNAzoXAxHXDhRxxUOpfpzpeZALTSh4m4QSEP1x'
    output_dir = "D:/du lieu o cu/HUTECH Courses/Social Networking Course/Data Crawling"  # Đường dẫn thư mục lưu file Excel

    collector = FacebookCollector(ACCESS_TOKEN)  # Khởi tạo đối tượng FacebookCollector với mã thông báo truy cập

    # Kiểm tra tính hợp lệ của mã thông báo
    if collector.check_token_validity():
        # Thu thập dữ liệu bài viết
        posts = collector.collect_data()

        # Lưu dữ liệu thu thập được vào file Excel
        if posts:
            collector.json_to_excel(posts, output_dir=output_dir)

if __name__ == "__main__":
    main()  # Chạy hàm main khi file được thực thi


FacebookCollector initialized successfully.
Invalid token: Error validating access token: Session has expired on Monday, 25-Nov-24 19:00:00 PST. The current time is Monday, 02-Dec-24 17:25:41 PST.


In [ ]:
def main():
    ACCESS_TOKEN = 'EAASkTBte5fYBO3KXGrlEjvU8DV7ttxDKnKK5ZApsW5YUBD8JpUXZCMhl8CZCPUOJMhPGn1GFLSov74f3b1Q8GjoINNhc0iC38newyP91Gom1kAJ0OmHOVND1mRnLVOL5VkY5aYw4VZA4ggyZCnI1aE9L2AEfLX6rnbIR19c73SFnZCWqXH9QOoXDaJONeJHQ8lB2m0PZCRcwesIoeDWNAzoXAxHXDhRxxUOpfpzpeZALTSh4m4QSEP1x'
    output_dir = "D:/du lieu o cu/HUTECH Courses/Social Networking Course/Data Crawling"

    collector = FacebookCollector(ACCESS_TOKEN)

    # Check if the token is valid
    if collector.check_token_validity():
        # Collect data
        posts = collector.collect_data()

        # Save collected data to an Excel file
        if posts:
            collector.json_to_excel(posts, output_dir=output_dir)

if __name__ == "__main__":
    main()


FacebookCollector initialized successfully.
Invalid token: Malformed access token
